### Statistical testing for ensemble simulations

This is a placeholder for an example describing how to perform statistical tests to determine whether a predicted growth rate is different in two ensemble FBA simulations.

In [2]:
import medusa